In [1]:
%pwd
%cd ../..

/home/rydevera3/data-science/umn_research/Deeplifting


In [12]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.models import DeepliftingSkipMLP
from deeplifting.optimization import deeplifting_predictions

In [26]:
problem = PROBLEMS_BY_NAME['bukin_n6']
dimensions = problem['dimensions']

# Set up the deeplifting model
input_size = 512
hidden_sizes = (128, 128)
output_size = dimensions
bounds = problem['bounds']
activation = 'sine'
output_activation = 'leaky_relu'
agg_function = 'sum'
trial = 0

model = DeepliftingSkipMLP(
    input_size=input_size,
    hidden_sizes=hidden_sizes,
    output_size=dimensions,
    bounds=bounds,
    skip_every_n=1,
    activation=activation,
    output_activation=output_activation,
    agg_function=agg_function,
    seed=trial,
)

# Need to setup an objective
results = np.zeros((1, 1, 3))
trial = 0

objective = problem['objective']
fn = lambda x: objective(x, results=results, trial=trial, version='pytorch')

In [27]:
# Put model into training model
model.train()

# Set up the optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

# Set up 10 epochs to start
for epoch in range(20):
    # Zero out the gradients
    optimizer.zero_grad()

    # Get the outputs of the model
    outputs = model(None)

    # Iterate over the objective function
    objective_values = []
    for i in range(len(outputs)):
        f = fn(outputs[i, :])
        objective_values.append(f)

    # Update f with the minimum information
    objective_values = torch.stack(objective_values)
    f = torch.min(objective_values)
    print(f'Objective value={f.detach().cpu().numpy()}')

    # Backward pass
    f.backward()

    # Next step of optimizer
    optimizer.step()

Objective value=2.7954373359680176
Objective value=1.620771050453186
Objective value=3.4175660610198975
Objective value=4.600964546203613
Objective value=3.809809684753418
Objective value=4.067779541015625
Objective value=5.22509765625
Objective value=2.967188596725464
Objective value=6.383037567138672
Objective value=2.2161412239074707
Objective value=7.040313720703125
Objective value=5.1415510177612305
Objective value=11.216092109680176
Objective value=3.850243091583252
Objective value=7.2703537940979
Objective value=10.243327140808105
Objective value=4.125312328338623
Objective value=5.20118522644043
Objective value=8.081161499023438
Objective value=18.159976959228516
